In [1]:
!nvidia-smi

Wed Apr 24 09:58:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [39]:
#@title Setup environment { display-mode: 'form' }
# %%capture
import os, subprocess, sys
from IPython import get_ipython

ipython = get_ipython()

if 'tensorboard' in ipython.extension_manager.loaded:
  %reload_ext tensorboard
else:
  %load_ext tensorboard

if 'autoreload' in ipython.extension_manager.loaded:
  %reload_ext autoreload
else:
  %load_ext autoreload

%autoreload 2

src_dir = os.path.abspath("/content/COMM061-NLP-CW")
first_install = False
if not os.path.exists(src_dir):
  output = subprocess.run(["git", "clone", "https://github.com/Cogniveon/COMM061-NLP-CW.git"], capture_output=True)
  print("Git clone:", output.stderr.decode('utf-8'))
  assert output.returncode == 0
  first_install = True

output = subprocess.run(
    ["git", "pull", "origin", "main"],
    cwd=src_dir,
    capture_output=True,
    # check=True,
)
print("Git pull:", output.stdout.decode('utf-8'))
assert output.returncode == 0


already_up_to_date = 'Already up to date.' in output.stdout.decode('utf-8')

if first_install or not already_up_to_date:
  output = subprocess.run(
      ["pip", "install", "-e", src_dir],
      capture_output=True,
      # check=True,
  )
  print("PIP install:", output.returncode)
  assert output.returncode == 0
else:
  print("Skipping install...")

Git pull: Updating c19c31e..d7d7afc
Fast-forward
 nlp_cw/trainer.py | 5 +++--
 1 file changed, 3 insertions(+), 2 deletions(-)

PIP install: 0


In [40]:
%env WANDB_PROJECT COMM061-NLP-CW
%env WANDB_LOG_MODEL end

env: WANDB_PROJECT=COMM061-NLP-CW
env: WANDB_LOG_MODEL=end


In [41]:
from nlp_cw import get_dataset, tokenize_dataset, init_trainer

dataset, id2label, label2id, label_list, num_labels = get_dataset(
    "surrey-nlp/PLOD-CW", ["B-O", "B-AC", "I-AC", "B-LF", "I-LF"]
)

dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 126
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 153
    })
})

In [42]:
# model_name = "romainlhardy/roberta-large-finetuned-ner"
model_name = "google-bert/bert-base-uncased"

tokenizer, tokenized_dataset = tokenize_dataset(dataset, model_name)

In [43]:
trainer = init_trainer(
    tokenizer,
    label_list,
    num_labels,
    id2label,
    label2id,
    batch_size=16,
    num_epochs=10,
    model_name_or_path=model_name,
    dataset=tokenized_dataset,
    push_to_hub=True,
    report_to="wandb",
)
trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.632500,0.262892,0.903591,0.908956,0.906265,0.904260
2,0.316900,0.229745,0.930936,0.913739,0.922257,0.918155
3,0.199400,0.228232,0.931017,0.919347,0.925145,0.922278
4,0.170000,0.219305,0.936616,0.928583,0.932582,0.927775
5,0.145700,0.234993,0.939494,0.937325,0.938408,0.933120
6,0.108600,0.243504,0.941793,0.934026,0.937893,0.933120
7,0.090800,0.253742,0.935661,0.928253,0.931942,0.927012
8,0.079100,0.267502,0.939032,0.934851,0.936937,0.931745


TrainOutput(global_step=536, training_loss=0.20418327646469003, metrics={'train_runtime': 332.6821, 'train_samples_per_second': 32.223, 'train_steps_per_second': 2.014, 'total_flos': 551939044151040.0, 'train_loss': 0.20418327646469003, 'epoch': 8.0})

In [37]:
trainer.evaluate()

{'eval_loss': 0.24625098705291748,
 'eval_precision': 0.9399469143994691,
 'eval_recall': 0.9345208642586178,
 'eval_f1': 0.9372260358944668,
 'eval_accuracy': 0.9323560848984578,
 'eval_runtime': 1.2783,
 'eval_samples_per_second': 98.57,
 'eval_steps_per_second': 6.258,
 'epoch': 9.0}

In [44]:
#@title Commit to HuggingFace { display-mode: 'form' }
commit_msg = 'google-bert-base-uncased-f1_937' #@param {type: "string"}
trainer.push_to_hub(commit_msg)

CommitInfo(commit_url='https://huggingface.co/cogniveon/nlpcw_bert-base-uncased-abbr/commit/f5e79b62885542d798456c67c92678f91236c6d5', commit_message='google-bert-base-uncased-f1_937', commit_description='', oid='f5e79b62885542d798456c67c92678f91236c6d5', pr_url=None, pr_revision=None, pr_num=None)